In [1]:
#haystack FAISS

import pandas as pd
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever


In [2]:

df = pd.read_csv('/home/karl/PycharmProjects/farm_haystack/filtered_df.csv')
df.reset_index(inplace=True)
df

,index,Time,cleaned_tweet,sent_analysis
0,0,2009-12-28 21:37:33+00:00,@user i think women who choose natural are like super heroes! especially sin...,"{'label': 'positive', 'score': 0.8187388181686401}"
1,1,2009-12-05 01:52:13+00:00,@user the bradley method is husband coached birth- where he takes the lead i...,"{'label': 'neutral', 'score': 0.9033392667770386}"
2,2,2009-12-01 18:43:35+00:00,my sister wanted to do a natural birth but she has now been in labor for 7 h...,"{'label': 'negative', 'score': 0.7766241431236267}"
3,3,2009-11-13 22:24:00+00:00,#haveababybyme u betta have health insurance cuz a natural birth and whatchu...,"{'label': 'negative', 'score': 0.6303679347038269}"
4,4,2009-11-09 00:56:08+00:00,my co-workers wife just gave natural birth to a 9.5 pound baby this am after...,"{'label': 'negative', 'score': 0.8341954946517944}"
...,...,...,...,...
431519,431519,2018-01-01 04:47:59+00:00,@user damn. i had a 24 hr labour but with the aid of a blessed epidural,"{'label': 'neutral', 'score': 0.37650564312934875}"
431520,431520,2018-01-01 03:07:46+00:00,@user @user @user speaking as someone who had her epidural installed incorre...,"{'label': 'negative', 'score': 0.49432462453842163}"
431521,431521,2018-01-01 01:28:34+00:00,@user well yeah if you don't have an epidural,"{'label': 'neutral', 'score': 0.7822378873825073}"
431522,431522,2018-01-01 01:09:31+00:00,@user yeah i’m definitely terrified of having babies at the moment but i’m g...,"{'label': 'negative', 'score': 0.6401897072792053}"


In [3]:
dicts = []
for i in df.itertuples():
    d = {
        'content': i[3],
        'meta': {
            'index': i[1],
            'sentiment': i[4],
            'time': i[2],
        }
    }
    dicts.append(d)


In [4]:
dicts[0]

{'content': '@user i think women who choose natural are like super heroes! especially since giving birth. epi though. caved bc of back labor.',
 'meta': {'index': 0,
  'sentiment': "{'label': 'positive', 'score': 0.8187388181686401}",
  'time': '2009-12-28 21:37:33+00:00'}}

In [5]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.save(index_path="my_faiss_index.faiss")

In [6]:
retriever = EmbeddingRetriever(document_store = document_store, embedding_model='distilroberta-base-msmarco-v2',model_format='sentence_transformers', use_gpu=True, batch_size=64)

/home/karl/mambaforge/envs/farm_haystack/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
document_store.write_documents(dicts)
document_store.update_embeddings(retriever=retriever)

Writing Documents:   0%|          | 0/431524 [00:00<?, ?it/s]

In [ ]:
def get_results(query, retriever, n_res = 1000):
    return [item for item in retriever.retrieve(query, top_k = n_res)]

In [ ]:
epidural_query = 'epidural labor'
epi_results = get_results(epidural_query, retriever)

In [ ]:
natural_query = 'natural labor'
natural_results = get_results(epidural_query, retriever)

In [ ]:
for r in epi_results:
    print(r.content, "|||" ,r.meta.sentiment)

In [ ]:
for r in natural_results:
    print(r.content,"|||",r.meta.sentiment)

In [ ]:
f = open('semantic_search_results.txt')

In [ ]:
for r in epi_results:
    f.write(r.content,"|||",r.meta.sentiment,'\n')

In [ ]:
for r in natural_results:
    f.write(r.content,"|||",r.meta.sentiment,'\n')